# IBM Applied Datascience Capstone- Project

# Introduction

In this project I decided to explore the restaurants in places that i want to visit. I decided to explore the cities of Oakland, Emeryville, and San Diego to get a better insight into potential suggestions for investors and entrepreneurs looking to make a sound investment in the restaurant business. I searched for what type of restaurant would be the best and in which city to focus on when scouting for potential property. The purpose of this project was to discern how accurately we can predict the amount of "likes" a new restaurant opening in the area can expect to have based on the type of cuisine it will serve and in which city in California it will open in. For this project I analyzed and modeled the data using machine learning by comparing both linear and logistic regressions to see which method yielded better predictive capabilities after training and testing.

# Data

The dataframe named 'raw_dataset' is the most complete compiled form that we can use for any analysis. It includes all the necessary details needed to perform machine learning on the data. After extracting the geographical coordinates of the cities: Oakland, Emeryville and San Diego I used the Foursquare API to get URLs and extract the raw data in JSON format. Each page had its own unique URL that was scraped for the columns: name, city, latitude, and longitude. The column 'city' helped sperate where restaurants are and I focused on those which were 1000km radius from the coordinates provided. Due to the big number of venue categories available from the API, we had to remove non-restaurant rows.The 'likes' data is needed to make the final decision. The 'id' column was used to get 'likes' using API and appended it into the dataframe. It was concluded by naming 'raw_dataset' which was used in the machine learning portion.

# Methodology

I used both linear and logistic regression to train and test the data. Linear regression will predict the number of 'likes' a new restaurant in this area will get. Sci-Kit Learn was used for this part.
Logistic regression was used as the classification method. Since, we used binning when classifying by number of 'likes', we essentially made use of multinomial logistic regression to perform our analysis. Although the ranges are discrete categories, they can be considered ordinal in nature. The logistic regression will need to be specified as being both multinomial and ordinal. Therefore Sci-Kit Learn package is used.

## Libraries

In [2]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
!pip install folium
import folium 
import matplotlib.pyplot as plt
import pylab as pl
import itertools
import warnings
warnings.filterwarnings('ignore')

from urllib.request import urlopen
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim 
from sklearn import linear_model
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score

print('All libraries have been imported')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.7 MB/s  eta 0:00:01
All libraries have been imported


## Foursquare city coordinates

In [3]:
address1 = 'Oakland, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))

address2 = 'Emeryville, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))

address3 = 'San Diego, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location3 = geolocator.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address3, latitude3, longitude3))

The geograpical coordinate of Oakland, California are 37.8044557, -122.2713563.
The geograpical coordinate of Emeryville, California are 37.8314089, -122.2865266.
The geograpical coordinate of San Diego, California are 32.7174202, -117.1627728.


In [31]:
CLIENT_ID = 'HPGIU0LLNQIUQV02H5ZH3TTX5FA1DZUNB5JAZAEQTXMD033O' # your Foursquare ID
CLIENT_SECRET = 'XICDZGDDOGGTDSTB3ACFDSUO1LVMFCGXJKJMZAVPUCNV0PKP' # your Foursquare Secret
VERSION = '20210611' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)


# create URLs
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)



print(url1, url2, url3)

Your credentails:
CLIENT_ID: HPGIU0LLNQIUQV02H5ZH3TTX5FA1DZUNB5JAZAEQTXMD033O
CLIENT_SECRET:XICDZGDDOGGTDSTB3ACFDSUO1LVMFCGXJKJMZAVPUCNV0PKP
https://api.foursquare.com/v2/venues/explore?&client_id=HPGIU0LLNQIUQV02H5ZH3TTX5FA1DZUNB5JAZAEQTXMD033O&client_secret=XICDZGDDOGGTDSTB3ACFDSUO1LVMFCGXJKJMZAVPUCNV0PKP&v=20210611&ll=37.8044557,-122.2713563&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=HPGIU0LLNQIUQV02H5ZH3TTX5FA1DZUNB5JAZAEQTXMD033O&client_secret=XICDZGDDOGGTDSTB3ACFDSUO1LVMFCGXJKJMZAVPUCNV0PKP&v=20210611&ll=37.8314089,-122.2865266&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=HPGIU0LLNQIUQV02H5ZH3TTX5FA1DZUNB5JAZAEQTXMD033O&client_secret=XICDZGDDOGGTDSTB3ACFDSUO1LVMFCGXJKJMZAVPUCNV0PKP&v=20210611&ll=32.7174202,-117.1627728&radius=1000&limit=100


## Data explore

In [5]:
# scrape the data from the generated URLs

results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

results3 = requests.get(url3).json()
results3

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# FIRST CITY   

venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = pd.json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 = nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

# SECOND CITY

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 = pd.json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 = nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

# THIRD CITY

venues3 = results3['response']['groups'][0]['items']
nearby_venues3 = pd.json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues3 = nearby_venues3.loc[:, filtered_columns3]

# filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]





print('{} venues for Oakland, California were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print()
print('{} venues for Emeryville, California were returned by Foursquare.'.format(nearby_venues2.shape[0]))
print()
print('{} venues for San Diego, California were returned by Foursquare.'.format(nearby_venues3.shape[0]))


100 venues for Oakland, California were returned by Foursquare.

100 venues for Emeryville, California were returned by Foursquare.

100 venues for San Diego, California were returned by Foursquare.


In [6]:
# add locations data to the data sets of each city

nearby_venues1['city'] = 'Oakland'
nearby_venues2['city'] = 'Emeryville'
nearby_venues3['city'] = 'San Diego'

# combine the three cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)
nearby_venues = nearby_venues.append(nearby_venues3)

nearby_venues

,name,categories,lat,lng,id,city
0,Oaklandish,Clothing Store,37.805075,-122.270726,4dfb9c2c1f6eeef806ab898c,Oakland
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland
2,Bar Shiru,Bar,37.806378,-122.270393,5c5b9abdf870fd002c35d291,Oakland
3,Cape & Cowl,Comic Shop,37.806725,-122.272747,56562410498ea43ab630819a,Oakland
4,Cafe Van Kleef,Bar,37.806660,-122.270273,46884818f964a52056481fe3,Oakland
5,Lucky Duck Bicycle Cafe,Café,37.801684,-122.268656,57bf810c498ee0a34d9f8ca1,Oakland
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland
7,Woods Bar & Brewery,Brewery,37.806889,-122.270415,5419f32c498e561ee5c2fa38,Oakland
8,Nature Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.802157,-122.270983,4f52deb2e4b0ac6d0c91df05,Oakland
9,Fox Theater,Music Venue,37.808136,-122.270670,4abc421df964a520ef8620e3,Oakland


In [7]:
nearby_venues['categories'].unique()

array(['Clothing Store', 'Vegetarian / Vegan Restaurant', 'Bar',
       'Comic Shop', 'Café', 'Japanese Restaurant', 'Brewery',
       'Music Venue', 'Vietnamese Restaurant', 'Bagel Shop',
       'American Restaurant', 'Seafood Restaurant', 'Skating Rink',
       'Kitchen Supply Store', 'Coffee Shop', 'Mexican Restaurant',
       'Beer Bar', 'Furniture / Home Store', 'Juice Bar', 'Dessert Shop',
       'Brazilian Restaurant', 'Afghan Restaurant', 'Chinese Restaurant',
       'Caribbean Restaurant', 'Lounge', 'Salad Place', 'Bakery',
       'Ice Cream Shop', 'Sausage Shop', 'Wine Bar', 'Taco Place',
       'Cocktail Bar', 'Climbing Gym', 'Sandwich Place',
       'Bubble Tea Shop', 'Food Court', 'Burger Joint', 'Tiki Bar',
       'New American Restaurant', 'Nightclub', 'Gay Bar',
       'Tapas Restaurant', 'Burmese Restaurant', 'Indian Restaurant',
       'Cambodian Restaurant', 'Falafel Restaurant', 'Museum',
       'Beer Garden', 'Street Food Gathering', 'ATM',
       'Southern / Soul 

## Data cleaning

In [8]:
# check list and manually remove all non-restaurant data

nearby_venues['categories'].unique()

removal_list = ['Clothing Store','Bar','Brewery', 
                'Comic Shop', 'Yoga Studio','Café', 
                'Coffee Shop', 'Tiki Bar', 'Music Venue', 
                'Wine Bar',  'Cocktail Bar', 'Dance Studio', 
                'Gym / Fitness Center','Beer Bar', 
                'Bubble Tea Shop', 'Nightclub', 'Food Court', 
                'Ice Cream Shop', 'Cupcake Shop', 'Skating Rink', 
                'Dessert Shop', 'Climbing Gym', 'Bakery', 
                'Farmers Market', 'Gay Bar','Beer Garden',
                'Tea Room','Arts & Crafts Store', 'Grocery Store', 
                'Sports Bar', 'Museum', 'Street Food Gathering', 
                'Library', 'Skate Park', 'Movie Theater','Park', 
                'Gym', 'Stadium', 'Furniture / Home Store', 'Discount Store', 
                'Playground', 'Cosmetics Shop', 'Casino', 
                'Pet Store','Electronics Store', 'Snack Place',
                'Salon / Barbershop', 'Shopping Plaza', 'Deli / Bodega', 
                'Candy Store', 'Liquor Store', 'Hotel', 
                'Shoe Store', 'Bookstore', 'Shopping Mall', 
                'Dive Bar', 'Video Game Store', 'Pharmacy', 
                'Accessories Store', 'Lingerie Store', 'Mobile Phone Shop', 
                'Pool Hall', 'Juice Bar', 'Kids Store', 
                'Supplement Shop', 'Big Box Store', 'Mattress Store', 
                'Hardware Store', 'Paper / Office Supplies Store', 'Theater', 
                'Business Service', 'Donut Shop', 'Beer Store', 
                'Lounge', 'Health Food Store', 'Pedestrian Plaza', 
                'Hookah Bar', 'Concert Hall', 'Chocolate Shop', 
                'Hostel', 'Convenience Store', 'Pub', 
                'Plaza', 'Comedy Club', 'Speakeasy', 
                'Tattoo Parlor', 'Massage Studio']

nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique().tolist()

['Vegetarian / Vegan Restaurant',
 'Japanese Restaurant',
 'Vietnamese Restaurant',
 'Bagel Shop',
 'American Restaurant',
 'Seafood Restaurant',
 'Kitchen Supply Store',
 'Mexican Restaurant',
 'Brazilian Restaurant',
 'Afghan Restaurant',
 'Chinese Restaurant',
 'Caribbean Restaurant',
 'Salad Place',
 'Sausage Shop',
 'Taco Place',
 'Sandwich Place',
 'Burger Joint',
 'New American Restaurant',
 'Tapas Restaurant',
 'Burmese Restaurant',
 'Indian Restaurant',
 'Cambodian Restaurant',
 'Falafel Restaurant',
 'ATM',
 'Southern / Soul Food Restaurant',
 'Breakfast Spot',
 'Jazz Club',
 'Flower Shop',
 'Diner',
 'Pizza Place',
 'Mediterranean Restaurant',
 'Video Store',
 'Scandinavian Restaurant',
 'Wings Joint',
 "Men's Store",
 'Bank',
 'Asian Restaurant',
 'Burrito Place',
 'Fast Food Restaurant',
 'Optical Shop',
 'Sushi Restaurant',
 'Fried Chicken Joint',
 'Food',
 'Trail',
 'Martial Arts School',
 'Food & Drink Shop',
 'French Restaurant',
 'Fondue Restaurant',
 'Theme Restauran

## Dataframe

In [9]:
# set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues.head()

[77, 64, 33, 72, 22, 7, 63, 50, 115, 199, 40, 66, 43, 100, 51, 0, 176, 182, 367, 33, 25, 15, 24, 0, 39, 237, 91, 171, 4, 27, 226, 38, 46, 9, 76, 1, 2, 42, 247, 13, 38, 66, 7, 191, 8, 54, 68, 136, 132, 332, 24, 4, 0, 1, 16, 36, 18, 54, 0, 1, 2, 31, 60, 17, 63, 92, 22, 68, 17, 4, 1, 15, 65, 7, 31, 0, 1, 7, 3, 0, 4, 156, 75, 33, 9, 15, 81, 18, 131, 26, 105, 59, 171, 41, 24, 38, 22, 21, 31, 30, 2, 298, 35, 31, 61, 7, 2, 132, 12, 222, 31, 480, 97, 187, 23, 34, 104, 152, 16, 140, 92, 19, 108, 50, 51, 54]


,name,categories,lat,lng,id,city,likes
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,64
8,Nature Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.802157,-122.270983,4f52deb2e4b0ac6d0c91df05,Oakland,33
10,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22


In [10]:
nearby_venues.count()

name          126
categories    126
lat           126
lng           126
id            126
city          126
likes         126
dtype: int64

In [11]:
# this is really the raw dataset now so let us rename it something more appropriate

raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,city,likes
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,64
8,Nature Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.802157,-122.270983,4f52deb2e4b0ac6d0c91df05,Oakland,33
10,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22


## Data for machine learning

In [12]:
# inspecting the raw dataset shows that there may be too many different types of cuisines

raw_dataset['categories'].unique().tolist()

['Vegetarian / Vegan Restaurant',
 'Japanese Restaurant',
 'Vietnamese Restaurant',
 'Bagel Shop',
 'American Restaurant',
 'Seafood Restaurant',
 'Kitchen Supply Store',
 'Mexican Restaurant',
 'Brazilian Restaurant',
 'Afghan Restaurant',
 'Chinese Restaurant',
 'Caribbean Restaurant',
 'Salad Place',
 'Sausage Shop',
 'Taco Place',
 'Sandwich Place',
 'Burger Joint',
 'New American Restaurant',
 'Tapas Restaurant',
 'Burmese Restaurant',
 'Indian Restaurant',
 'Cambodian Restaurant',
 'Falafel Restaurant',
 'ATM',
 'Southern / Soul Food Restaurant',
 'Breakfast Spot',
 'Jazz Club',
 'Flower Shop',
 'Diner',
 'Pizza Place',
 'Mediterranean Restaurant',
 'Video Store',
 'Scandinavian Restaurant',
 'Wings Joint',
 "Men's Store",
 'Bank',
 'Asian Restaurant',
 'Burrito Place',
 'Fast Food Restaurant',
 'Optical Shop',
 'Sushi Restaurant',
 'Fried Chicken Joint',
 'Food',
 'Trail',
 'Martial Arts School',
 'Food & Drink Shop',
 'French Restaurant',
 'Fondue Restaurant',
 'Theme Restauran

In [13]:
# we can group some cuisines together to make a better categorical variable

european = ['Mediterranean Restaurant', 'Scandinavian Restaurant', 'Pizza Place',
       'French Restaurant', 'Falafel Restaurant', 'Italian Restaurant',
       'Turkish Restaurant']

latin = ['Mexican Restaurant', 'Taco Place', 'Brazilian Restaurant', 
          'Burrito Place']

asian = ['Japanese Restaurant', 'Vietnamese Restaurant', 'Chinese Restaurant',
         'Hot Dog Joint', 'Hotpot Restaurant', 'Indian Restaurant',
         'Thai Restaurant', 'Dumpling Restaurant', 'Dim Sum Restaurant',
         'Asian Restaurant', 'Filipino Restaurant', 'Sushi Restaurant',
         'Ramen Restaurant']

american = ['Vegetarian / Vegan Restaurant', 'Seafood Restaurant', 'Caribbean Restaurant',
           'Burger Joint', 'American Restaurant', 'New American Restaurant',
            'Southern / Soul Food Restaurant', 'Diner']

casual = ['Bagel Shop', 'Sandwich Place', 'Fried Chicken Joint', 
          'Breakfast Spot', 'Wings Joint', 'Fast Food Restaurant',
          'Theme Restaurant']

def conditions(s):
    if s['categories'] in european:
        return 'european'
    if s['categories'] in latin:
        return 'latin'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in casual:
        return 'casual'

raw_dataset['categories_classified'] = raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77,american
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,64,asian
8,Nature Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.802157,-122.270983,4f52deb2e4b0ac6d0c91df05,Oakland,33,american
10,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72,asian
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22,casual
12,Catered To You,American Restaurant,37.807046,-122.270191,4be210911dd22d7f354493bd,Oakland,7,american
13,The Cook And Her Farmer,Seafood Restaurant,37.801583,-122.274860,5376a879498e8eeb2402cd71,Oakland,63,american
14,Binh Minh Quan,Vietnamese Restaurant,37.802020,-122.269396,4a75022cf964a52043e01fe3,Oakland,50,asian
16,Umami Mart,Kitchen Supply Store,37.800483,-122.273744,501c7838e4b08947b4df755e,Oakland,115,None
19,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Oakland,199,latin


In [14]:
# double check to make sure categories_classified has been created correctly

pd.crosstab(index = raw_dataset["categories_classified"],
            columns="count")

col_0,count
categories_classified,
american,27
asian,23
casual,17
european,21
latin,15


In [15]:
raw_dataset['likes'].mean()

65.9920634920635

In [16]:
# create a function to bin for us

def rankings(df):
    
    if df['likes'] <= 60:
        return 3
    
    elif df['likes'] <= 100:
        return 2
    
    elif df['likes'] > 100:
        return 1

In [17]:
# apply rankings function to dataset

raw_dataset['ranking'] = raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified,ranking
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Oakland,77,american,2
6,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Oakland,64,asian,2
8,Nature Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.802157,-122.270983,4f52deb2e4b0ac6d0c91df05,Oakland,33,american,3
10,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Oakland,72,asian,2
11,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Oakland,22,casual,3
12,Catered To You,American Restaurant,37.807046,-122.270191,4be210911dd22d7f354493bd,Oakland,7,american,3
13,The Cook And Her Farmer,Seafood Restaurant,37.801583,-122.274860,5376a879498e8eeb2402cd71,Oakland,63,american,2
14,Binh Minh Quan,Vietnamese Restaurant,37.802020,-122.269396,4a75022cf964a52043e01fe3,Oakland,50,asian,3
16,Umami Mart,Kitchen Supply Store,37.800483,-122.273744,501c7838e4b08947b4df755e,Oakland,115,None,1
19,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Oakland,199,latin,1


## Linear Regression (machine learning)

In [18]:
# create dummies for linear regression modelling

# one hot encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'city',]], 
                               prefix="", 
                               prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]


reg_dataset.head()

,name,american,asian,casual,european,latin,Emeryville,Oakland,San Diego,ranking,likes
1,Golden Lotus Vegetarian Restaurant,1,0,0,0,0,0,1,0,2,77
6,Abura-Ya,0,1,0,0,0,0,1,0,2,64
8,Nature Vegetarian Restaurant,1,0,0,0,0,0,1,0,3,33
10,Tay Ho Restaurant & Bar,0,1,0,0,0,0,1,0,2,72
11,Beauty’s Bagel Shop,0,0,1,0,0,0,1,0,3,22


In [19]:
# Multiple Linear Regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y = np.asanyarray(train[['likes']])
regr.fit (x, y)

# The coefficients

print ('Coefficients: ', regr.coef_)

Coefficients:  [[ 53.01131744 -11.22866919   3.20879624   0.06561155  26.69453521
    5.12724709 -14.92719052   9.79994343]]


In [20]:
# Multiple Linear Regression Prediction Capabilities

y_hat= regr.predict(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

x = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y = np.asanyarray(test[['likes']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 14292.81
Variance score: -0.03


## Logistic regression

In [21]:
# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Oakland', 
                         'Emeryville', 'San Diego']])

y_test = np.asanyarray(test['ranking'])


mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[ 0.47420673 -0.877357   -0.06661068 -0.12425692  0.13761019 -0.18396561
 -0.15971042  0.34367645]


In [22]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob


# average = None, average = 'micro', average = 'macro', or average = 'weighted'
jaccard_score(y_test, yhat, average='weighted')

0.2962962962962963

In [23]:
log_loss(y_test, yhat_prob)

1.1203002254268557

In [24]:
# Exploration of Coefficient Magnitudes of Full Dataset

x_all = np.asanyarray(reg_dataset[['american', 'asian', 'casual',
                                   'european', 'latin', 'Oakland', 
                                   'Emeryville', 'San Diego']])

y_all = np.asanyarray(reg_dataset['ranking'])



LR = linear_model.LogisticRegression(multi_class='multinomial',
                                            solver='newton-cg',
                                            fit_intercept=True).fit(x_all,
                                                                    y_all)

LR

coef = LR.coef_[0]
print(coef)

[ 0.38812992 -0.50659868 -0.21538495 -0.01788764  0.21465441  0.02242994
 -0.38888171  0.36643179]


In [25]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00         1
           3       0.60      0.75      0.67        16

    accuracy                           0.44        27
   macro avg       0.20      0.25      0.22        27
weighted avg       0.36      0.44      0.40        27



# Results

A linear regression model was trained on a random subsample of 80% and then the other 20% was used for testing purposes. In order to evaluate if the model is reasonable, the residual sum of squares and variance score were both calculated (14292.81, -0.03). The variance score is quite low, which means that is not a good way of modeling our data. Therefore, we moved on to logistic regression for our analysis.
The multinomial ordinal logistic regression model was also trained on a random subsample of 80% and then tested on the remaining 20%. The jaccard score and log-loss were both calculated (29.62% and 1.12 respectively). Although the prediction is not promosing, a jaccard score of 29% is somewhat reasonable. The classification report is included in the analysis.
Given the modestly accurate ability of this mode, we have the ability to run the model on the complete dataset. The coefficients we got show that opening a restaurant in Emeryville, or serving cuisine that is asian, or casual, are negatively associated with 'likes'.

# Decision

The first thing to note is that given the data, logistic regression presents a better fit for the data over linear regression. Using logistic regression we were able to obtain a Jaccard Score of 29.62%, which although not perfect, is more reasonable than the low variance score obtained from the linear regression. As stated before, please note that for the purposes of this project, we are assumming that likes are a good proxy for how well a new restaurant will do in terms of brand, image and by extension how well the restaurant will perform business-wise. Whether or not these assumptions hold up in a real-life scenario is up for discussion, but this project does contain limitations in scope due to the amount of data that can be fetched from the FourSquare API.
As such, to obtain insights into this data, we can proceed with breaking down the results of the logistic regression model. The results showed that the precision score for classifying whether the new restaurant would fall into classes 1, 2, or 3 (highest, medium, lowest) were 0%, 0%, and 60%. Therefore, the model is better at predicting if a restaurant will fall into the best or worst percentile of likes. This is good as we are mostly concerned with whether the restuarant will perform well or not so the high accuracy of predictions for the two extremum is a welcome feature. This allows us to fairly accurately predict the general performance of the business opportunity. Different binning methods for the classes were attempted, but the use of 3 bins by far yielded the best Jaccard Score.
Additionally, not only are we attempting to predict the general business performance but also pull insights to inform on business strategy. In this case strategy insight can be gleamed from the coefficient values from running the logistic regression on the full dataset. As such, we can see that opening a restaurant in Emeryville, or serving cuisine that is asian or casual in nature, are associated negatively with "likes." This suggests that the business opportunity should be opening a restaurant in either Oakland or San Diego, with a cuisine that is European, Latin, or American in nature would be the best approach for maximizing likes.

# Conclusion

In conclusion, after analyzing restaurant 'likes' in California from the 300 restaurants, we can conclude that the approach to best take when looking to maximize business performance (as measured by 'likes') is to open a restaurant that is either European, Latino, or American and that opening the venue in either Oakland or San Diego rather than Emeryville would be the best approach. Additionally, the predictive capabilities of the logistic regression prediction model proved to be the most accurate for classifying whether a restaurant fell in either the best or worst classes when the data was binned into their 3 respective classes.